In [9]:
import pandas as pd
import numpy as np
import re
import datetime

In [7]:
filePath = "/home/scio/KakaoTalk_20190310_0154_03_043_group.txt"
inputFile = open(filePath,'r')

In [4]:
reDate = re.compile('--------------- \d{4}년 \d{1,2}월 \d{1,2}일.*')
reJoin = re.compile('.*님이 들어왔습니다.')
reGetOut = re.compile('.*님이 나갔습니다.')
reChat = re.compile('\[.*\] 오.+ \d{1-2}:\d{1-2} .*')

In [10]:
all = []
for line in inputFile:
    all.append(line)

inAndOutList = []
chatList = []

for idx, line in enumerate(all):
    
    if len(line) == 1:
        continue
    
    else:
        checkerDate = reDate.match(line)
        if checkerDate is not None: # 몇년몇월몇일이 시작되었음을 알리는 라인인 경우
            year, month, day, weekDay = re.split(' ',str(checkerDate))[5:9]
            __datetime = year+month+day
            convert_datetime = datetime.datetime.strptime(__datetime, "%Y년%m월%d일").date()
            # print(convert_datetime)
            continue
        
        checkerJoin = reJoin.match(line) # 방에 들어온 사람 체크
        if checkerJoin is not None:
            nickName = re.split("'",str(checkerJoin))[1][:-10]
            inAndOutList.append([1, nickName, convert_datetime]) # In일 경우 1
            continue
            # print('In --------', nickName, convert_datetime)
    
        checkerGetOut = reGetOut.match(line) # 방에서 나간 사람 체크
        if checkerGetOut is not None:
            nickName = re.split("'",str(checkerGetOut))[1][:-9]
            inAndOutList.append([0, nickName, convert_datetime]) # In일 경우 1
            continue
            # print('GetOut --------', nickName, convert_datetime)
        
        else: #위 경우가 아닌 경우는 채팅내용 뿐
            # temp = re.split("]",str(line))
            try:
                nickName, time, text = re.split("]",str(line))
                nickName, time = nickName[1:], time[2:]
                # print(time[:2])
                
                tempTime = re.split('[ :]',time)
                tempTime[1] = int(tempTime[1])
                tempTime[2] = int(tempTime[2])
                
                if tempTime[0] == '오후':
                    tempTime[1] =  12 + tempTime[1]
                
                chatTime = str(tempTime[1])+'시'+str(tempTime[2])+'분'
                convert_chatTime = datetime.datetime.strptime(chatTime, "%H시%M분").time()
                
                chatList.append([nickName, tempTime[0], convert_datetime, convert_chatTime, text])
                # print(nickName, time, text)
            except:
                # 글 내에 엔터가 있을 경우 엔터 아래의 글 부분에서 오류가 남. 이부분은 나중에 해결하기로!
                pass

In [12]:
dfChatList = pd.DataFrame(chatList)
dfChatList.columns = ['NickName', 'AMPM', 'Date', 'Time', 'Text']

dfInAndOutList = pd.DataFrame(inAndOutList)
dfInAndOutList.columns = ['IsIn', 'NickName', 'Date']

ValueError: Length mismatch: Expected axis has 0 elements, new values have 5 elements